## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Existing Agent Example

This notebook demonstrates working with pre-existing Azure AI Agents by providing agent IDs, showing agent reuse patterns for production scenarios.

## Features Covered:
- Creating and managing persistent Azure AI agents
- Working with existing agent IDs
- Agent lifecycle management (create, use, delete)
- Production patterns for agent reuse
- Manual cleanup of agents

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with an existing Azure AI agent by connecting to it using its agent ID.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [1]:
import os
from pathlib import Path
from random import randint
from typing import Annotated
from agent_framework import AgentRunResponse, ChatResponseUpdate, HostedCodeInterpreterTool
from agent_framework.azure import AzureAIAgentClient
from azure.ai.agents.models import (
    RunStepDeltaCodeInterpreterDetailItemObject,
)
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [2]:
# This cell is for demonstration purposes only - actual async client setup is in the main() function
# The main() function uses the proper async imports from cell 4
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file


# Get configuration from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

# Validate and clean endpoint (remove any inline comments)
if project_endpoint and '#' in project_endpoint:
    print("⚠️  WARNING: Endpoint contains a '#' character - cleaning it...")
    project_endpoint = project_endpoint.split('#')[0].strip()

print(f"🔗 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment}")

print("📝 Note: The actual client initialization happens in the main() function using async context managers")
print("✅ Environment variables and async imports are ready for use")

🔗 Project Endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
🤖 Model Deployment: gpt-4o
📝 Note: The actual client initialization happens in the main() function using async context managers
✅ Environment variables and async imports are ready for use


## Check Environment Variables

Let's verify that the required environment variables are set:

## Define Function Tools

Let's define a simple weather function that our agent can use:

In [3]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Create and Use Existing Agent

This example shows how to:
1. Create an agent that persists beyond the current session
2. Use the agent ID to work with an existing agent
3. Properly clean up resources

In [4]:
async def main() -> None:
    """Example showing how to create and use a persistent agent."""
    print("=== Azure AI Agent with Existing Agent Example ===")

    # For authentication, run `az login` command in terminal
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as chat_client,
    ):
        # Create a persistent agent (without using context manager so it doesn't auto-delete)
        agent = chat_client.create_agent(
            name="WeatherAgent-Persistent",
            instructions="You are a helpful weather agent.",
            tools=get_weather,
        )
        
        print(f"✅ Created agent with ID: {agent.id}")
        print(f"📝 This agent will persist in Azure AI Foundry")

        try:
            # Use the agent
            query = "What's the weather like in Tokyo?"
            print(f"\nUser: {query}")
            result = await agent.run(query)
            print(f"Agent: {result.text}\n")
            
        finally:
            # The agent is NOT automatically deleted here since we didn't use it as a context manager
            # To delete manually, you would need to call the delete API
            print(f"💡 Agent with ID {agent.id} is still available in Azure AI Foundry")
            print(f"   You can find it in the Azure AI Studio portal or reuse it by ID")

## Execute the Example

Run the main function to see the existing agent workflow:

In [5]:
# Run the main function
await main()

=== Azure AI Agent with Existing Agent Example ===
✅ Created agent with ID: 56348059-a1f3-4ae9-a211-8ba6fb05247d
📝 This agent will persist in Azure AI Foundry

User: What's the weather like in Tokyo?


[2026-02-11 15:31:30 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Agent: The weather in Tokyo is currently stormy with a high temperature of 13°C. Stay safe and plan accordingly!

💡 Agent with ID 56348059-a1f3-4ae9-a211-8ba6fb05247d is still available in Azure AI Foundry
   You can find it in the Azure AI Studio portal or reuse it by ID


## Working with Agent from Portal

Here's an example of how to work with an agent that was created through the Azure AI Studio portal:

In [6]:
async def use_portal_agent_example():
    """Example showing how to use an agent created in the Azure AI Studio portal."""
    print("=== Using Agent from Portal Example ===")
    
    # If you have an agent ID from the portal, you can use it directly
    # Replace 'your-agent-id-here' with the actual agent ID from the portal
    portal_agent_id = "asst_2XKr7UoD1qZigI8m5oEfFA3Z"  # Get this from Azure AI Studio
    
    print(f"Note: To use a portal agent, replace the agent ID with your actual agent ID from Azure AI Studio")
    print(f"Example agent ID format: {portal_agent_id}")
    print("\nTo use a portal agent, uncomment and modify the following code:")
    print("""    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            agent_id=portal_agent_id  # Your agent ID here
        ).get_agent() as agent,
    ):
        result = await agent.run("What's the weather like in London?")
        print(f"Portal Agent Result: {result.text}")
    """)
    
# Run the portal agent example
await use_portal_agent_example()

=== Using Agent from Portal Example ===
Note: To use a portal agent, replace the agent ID with your actual agent ID from Azure AI Studio
Example agent ID format: asst_2XKr7UoD1qZigI8m5oEfFA3Z

To use a portal agent, uncomment and modify the following code:
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            agent_id=portal_agent_id  # Your agent ID here
        ).get_agent() as agent,
    ):
        result = await agent.run("What's the weather like in London?")
        print(f"Portal Agent Result: {result.text}")
    


## Advanced Example: Persistent Agent Management

This example shows how to create, list, and manage multiple agents:

In [7]:
async def agent_management_example():
    """Example showing how to list and reuse agents."""
    print("=== Agent Management Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            async_credential=credential,
            project_endpoint=project_endpoint,
            model_deployment_name=model_deployment
        ) as chat_client,
    ):
        print("\n📝 Creating a reusable weather agent...")
        
        # Create a persistent agent
        agent = chat_client.create_agent(
            name="ReusableWeatherAgent",
            instructions="You are a persistent weather agent that provides detailed weather information.",
            tools=get_weather,
        )
        print(f"✅ Created agent with ID: {agent.id}")
        
        try:
            # Use the agent multiple times
            queries = [
                "What's the weather in New York?",
                "How about in Paris?",
                "Compare the weather in Tokyo and Sydney"
            ]
            
            for query in queries:
                print(f"\n🤔 User: {query}")
                result = await agent.run(query)
                print(f"🤖 Agent: {result.text}")
                
        finally:
            print(f"\n💡 Agent with ID {agent.id} is still available for reuse")
            print(f"   This agent can be accessed from Azure AI Studio portal")

# Run the agent management example
await agent_management_example()

=== Agent Management Example ===

📝 Creating a reusable weather agent...
✅ Created agent with ID: 12456b83-739d-4a1e-b105-a6e51edfb8de

🤔 User: What's the weather in New York?


[2026-02-11 15:32:04 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: The weather in New York is currently cloudy with a high of 22°C. Let me know if you'd like more detailed information!

🤔 User: How about in Paris?


[2026-02-11 15:32:10 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: The current weather in Paris is cloudy with a high of 12°C. Let me know if you'd like more details!

🤔 User: Compare the weather in Tokyo and Sydney


[2026-02-11 15:32:15 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: Currently, in Tokyo, the weather is stormy with a high temperature of 21°C, whereas Sydney is experiencing sunny weather with a high of 10°C.

💡 Agent with ID 12456b83-739d-4a1e-b105-a6e51edfb8de is still available for reuse
   This agent can be accessed from Azure AI Studio portal


## Key Takeaways

1. **Persistent Agents**: Agents can be created to persist beyond a single session, useful for production scenarios
2. **Agent ID Management**: You can work with agents using their IDs, whether created programmatically or through the portal
3. **Resource Management**: Always clean up agents when they're no longer needed to avoid unnecessary costs
4. **Production Patterns**: This approach is suitable for production applications where agents need to be reused
5. **Portal Integration**: Agents created in Azure AI Studio can be used directly in code
6. **Client Flexibility**: You can create agents with or without providing a project client

## Best Practices

1. **Environment Variables**: Use environment variables for configuration to keep credentials secure
2. **Error Handling**: Always use try-finally blocks to ensure agent cleanup
3. **Agent Naming**: Use descriptive names for agents to make management easier
4. **Documentation**: Keep track of agent IDs and their purposes
5. **Resource Monitoring**: Monitor your Azure AI resource usage when working with persistent agents

## Use Cases

- **Production Applications**: Where agents need to persist across multiple user sessions
- **Team Collaboration**: Sharing agents across team members
- **Multi-Session Conversations**: Maintaining agent state across sessions
- **Agent Libraries**: Creating reusable agents for common tasks
- **Integration with Azure AI Studio**: Using agents created through the web interface